In [1]:
import utils.message_constructor as mc
import utils.vlm_utils as vu
import torch
import json

/home/pengzy/mambaforge/envs/qwen2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-07-28 16:39:33,548] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio
collect2: error: ld returned 1 exit status


 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.


/home/pengzy/mambaforge/envs/qwen2/lib/python3.11/site-packages/deepspeed/runtime/zero/linear.py:47: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @autocast_custom_fwd
/home/pengzy/mambaforge/envs/qwen2/lib/python3.11/site-packages/deepspeed/runtime/zero/linear.py:66: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @autocast_custom_bwd


In [2]:
json_path = r"./sln/VQA_SA/VQA-SA-question.json"
llm_base = r"./sln/qwen2.5_lm_7B"
device = "cuda:1"

In [3]:
vu.ensure_vl_base(
    base_path=llm_base,
    device=device,
    torch_dtype=torch.bfloat16,
)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Loading checkpoint shards: 100%|██████████| 4/4 [00:11<00:00,  2.92s/it]


vlm based on <class 'transformers.models.qwen2.modeling_qwen2.Qwen2ForCausalLM'> is loaded


In [4]:
llm_model = vu.vl_model
llm_processor = vu.vl_processor

def qwen25_infer(
        messages:mc.Message,
        max_new_tokens:int=vu.DEFAULT_MAX_NEW_TOKENS
    ) -> str:
    text = llm_processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = llm_processor(text=[text], return_tensors="pt")
    inputs = inputs.to(llm_model.device)
    generated_ids = llm_model.generate(**inputs, max_new_tokens=max_new_tokens)
    generated_ids_trimmed = [out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)]
    output_text = llm_processor.batch_decode(generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False)
    return "".join(output_text)

In [20]:
%load_ext autoreload
%autoreload 2
import utils.prompts as sp

question = "从摄影角度看，茶壶在吊灯的哪个方向？请从前方、后方、左方、右方、上方、下方里选择一个方向"
answer = "### 推理过程：\n\n1. **观察吊灯的位置**：\n   - 吊灯位于图片的中央偏上位置，悬挂在天花板上。它是一个圆锥形的灯具，由木质材料制成。\n\n2. **观察茶壶的位置**：\n   - 茶壶位于图片的中央偏下位置，放置在圆形餐桌的中间区域。\n\n3. **确定吊灯与茶壶的相对位置**：\n   - 从图片中可以看出，吊灯在茶壶的正上方。吊灯悬挂在天花板上，而茶壶则放在地面上的餐桌上。\n\n4. **分析选项**：\n   - 前方：吊灯不在茶壶的前方。\n   - 后方：吊灯不在茶壶的后方。\n   - 左方：吊灯不在茶壶的左方。\n   - 右方：吊灯不在茶壶的右方。\n   - 上方：吊灯在茶壶的正上方。\n   - 下方：吊灯不在茶壶的下方。\n\n### 结论：\n根据上述分析，吊灯在茶壶的正上方。\n\n**答案：上方**"

print(sp.RECTIFY_SYSTEM)

message = mc.Message().add_role_block("system", sp.RECTIFY_SYSTEM)\
                .add_role_block("user", f"回答：{answer}\n问题：{question}")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
我们需要对一些问答做出修改，请仔细判断给出的回答是否解答了给出的问题，若没有，则根据回答内容修改回答，使答案直接回答给出的问题。

示例1：

问题：从摄影角度看，茶壶在吊灯的哪个方向？请从前方、后方、左方、右方、上方、下方里选择一个方向
回答：### 推理过程：

1. **观察吊灯的位置**：
   - 吊灯位于图片的中央偏上位置，悬挂在天花板上。它是一个圆锥形的灯具，由木质材料制成。

2. **观察茶壶的位置**：
   - 茶壶位于图片的中央偏下位置，放置在圆形餐桌的中间区域。

3. **确定吊灯与茶壶的相对位置**：
   - 从图片中可以看出，吊灯在茶壶的正上方。吊灯悬挂在天花板上，而茶壶则放在地面上的餐桌上。

4. **分析选项**：
   - 前方：吊灯不在茶壶的前方。
   - 后方：吊灯不在茶壶的后方。
   - 左方：吊灯不在茶壶的左方。
   - 右方：吊灯不在茶壶的右方。
   - 上方：吊灯在茶壶的正上方。
   - 下方：吊灯不在茶壶的下方。

### 结论：
根据上述分析，吊灯在茶壶的正上方。

**答案：上方**

修改：答案：下方
修改原因：问题中提问茶壶在吊灯的哪个方向，而不是吊灯在茶壶的哪个方向，回答中颠倒了对象的顺序，得到了错误答案，故需要将两者的位置关系反转得到正确答案；




In [21]:
output = qwen25_infer(message, 2048)
print(output)

修改后的答案：下方

解释：问题询问的是茶壶相对于吊灯的方向，而原始推理和结论中已经明确指出吊灯在茶壶的正上方。因此，从摄影角度看，茶壶在吊灯的下方。
